<a href="https://colab.research.google.com/github/SPThole/LTFS_repo/blob/main/LETSEE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting important libraies

In [ ]:
!pip install catboost
!pip install shap
!pip install catboost
!pip install optuna

     |████████████████████████████████| 65.8MB 57kB/s 
     |████████████████████████████████| 358kB 6.0MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp36-cp36m-linux_x86_64.whl size=489394 sha256=ebdc51c0a83a8bf124224bf2fa0758aacacb341335a71b9f08bb7dc8f6ac36c0
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap
     |████████████████████████████████| 296kB 5.8MB/s 
     |████████████████████████████████| 1.1MB 8.0MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 481kB 16.8MB/s 
     |████████████████████████████████| 143kB 21.4MB/s 
     |████████████████████████████████| 112kB 20.1MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=7e6e52a7851a8942e55aa73b86e89344bb7cf8ac98869a9bbc875dc951f3549b
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import xgboost as xgb
import catboost as cat
import shap
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import optuna
import matplotlib.pyplot as plt
import zipfile 
import textwrap
import datetime
from tqdm import tqdm_notebook as tq
%matplotlib inline

# Helper Functions

In [ ]:
def same_ids(df,df1):
  a = set(df['ID'].values)
  b = set(df1['ID'].values)
  return len(set(a).intersection(set(b)))

In [ ]:
#Check if same id's lie in test or not
class preprocess:

  def __init__(self, df):
    """
    """
    self.df = df.copy()

  def de_dupe(self):
    print('Dropping Duplicates')
    print(f'Before Shape: {df.shape}')
    df = df.drop_duplicates()
    print(f'After Shape: {df.shape}')
    return df

 
  def replacer(self,xx):
    return [float(x) if x!='' else 0 for x in xx]

  
  def splitcomma(self,df,col):
    df[col] = df[col].apply(lambda x: str(x)[:-1] if str(x)[-1]==',' else x)
    df[col] = df[col].apply(lambda x: str(x).split(','))
    df[col] = df[col].apply(lambda x: self.replacer(x))
    return df[col]

  def history_data(self,cols_list,wrap_cols):
    print('PREPROCESSING HISTORY DATA')
    # ['REPORTED DATE - HIST','CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
    for col in cols_list:
      print(col)
      self.df[col] = self.splitcomma(self.df,col)
    for wraps in wrap_cols:
      self.df[wraps] = self.df[wraps].apply(lambda x:\
                                                      textwrap.wrap(str(x), 3))
                                                    
  def amountclean(self,x):
    x = str(x).split('/')[0]
    x = x.split(',')
    x = float(''.join(x))
    return x
    
  def amount_data(self,col_list):
    print('PREPROCESSING AMOUNT DATA')
    # ['CREDIT-LIMIT/SANC AMT','DISBURSED-AMT/HIGH CREDIT',
    # 'INSTALLMENT-AMT','CURRENT-BAL','OVERDUE-AMT','WRITE-OFF-AMT']
    for col in col_list:
      print(col)
      self.df[col] = self.df[col].apply(lambda x: self.amountclean(x)).astype('float64')

  def installments(self,col_list):
    print('PREPROCESSING INSTALLMENTS')
    for col in col_list:
      print(col)
      self.df[col] = self.df[col].apply(lambda x: self.amountclean(x))
      self.df[col+'_time'] = self.df[col].apply(\
                                      lambda x : str(x).split('/')[-1])
      
  def datenan(self,x):
    try:
        return datetime.datetime.strptime(x, '%Y%m%d')
    except:
        return pd.NaT
  
  def listofdates(self,df,col):
    df[col] = df[col].apply(lambda x: str(x)[:-1] if str(x)[-1]==',' else str(x))
    df[col] = df[col].apply(lambda x: str(x).split(','))
    df[col] = df[col].apply(lambda x: [self.datenan(i) for i in x ])
    return df[col]

  def date_data(self,col_list,date_lists):
    print('PREPROCESSING DATE DATA')
    # ['DATE-REPORTED','DISBURSED-DT',
    #   'CLOSE-DT','DISBURSED-DT']
    for col in col_list:
      print(col)
      self.df[col] = pd.to_datetime(self.df[col],errors='coerce')
    for col in date_lists:
      print(col)

      # 'REPORTED DATE - HIST'
      self.df[col] = self.listofdates(self.df, col)

  def cat_data(self,dfs,col_list):
      print('PREPROCESSING CATEGORICAL DATA')
      

      encoder = LabelEncoder() 
      for col in col_list:
        print(col)
        self.df[col] = self.df[col].fillna('UNK')
        dfs[col] = dfs[col].fillna('UNK')
        dfm = self.df[col].append(dfs[col], ignore_index = True)
        encoder = encoder.fit(dfm)
        self.df[col] = encoder.transform(self.df[col])
        dfs[col] = encoder.transform(dfs[col])



  def merger(self, df_left ,df_right, on_what ,how_to):

    assert df_left[on_what].dtype == df_right[on_what].dtype
    df_left, df_right = self.de_dupe((df_left, df_right))
    print(f'SHAPES BEFORE MERGE: {df_left.shape, df_right.shape}')
    dfm = df_left.merge(df_right, on = on_what, how = how_to)
    print(f'SHAPES AFTER MERGE: {dfm.shape}')
    return dfm

  def getmerged(self):

    self.trainer = self.merger(self.d_train, self.b_train, 'ID', 'inner')
    self.tester = self.merger(self.d_test, self.b_test, 'ID', 'inner')

  

  

  
    
    



In [ ]:
class lookin:

  def __init__(self, df):

    self.df = df

  def dtype_obj(self):

    objs_list = self.df.select_dtypes(['object']).columns
    self.objs_cols = list(objs_list)
    return list(objs_list)

  def unique_objs(self):
    lister = self.dtype_obj()
    self.objs_nunique = pd.DataFrame(self.df[lister].nunique()).\
                                  sort_values(by=0, ascending = False)
    return self.objs_nunique

  def na_check(self):
    dff = pd.DataFrame(self.df.isna().sum()).sort_values(by = 0,\
                                                         ascending = False)
    self.nas = dff[dff[0]!=0]

  def granularity(self, on_what):

    dff = pd.DataFrame(self.df[on_what].value_counts())
    print(dff.head())
    print(dff.iloc[:,0].max, dff.iloc[:,0].min())


  def run_this(self):

    self.unique_objs()
    self.na_check()

In [ ]:
def targetencode(dftrain,dftest,x,trgt,ag):
  dff = dftrain.copy()
  dff1 =  dftest.copy()
  for col in x:
    print(col)
    dff = dftrain[[col,trgt]]
    dffm = dftrain.groupby(col).agg({trgt:ag}).reset_index()
    dd = pd.Series(dffm[trgt].values,index=dffm[col].values).to_dict()
    dftest[col] = dff1[col].map(dd)
    dftrain[col] = dff[col].map(dd)
  return dftrain, dftest

    


# Getting data

In [ ]:


zipfiles = ['/content/drive/MyDrive/Test.zip', 
            '/content/drive/MyDrive/Train.zip']
target_path = '/content/drive/MyDrive'

for files in zipfiles:
  with zipfile.ZipFile(files,"r") as zip_ref:
      zip_ref.extractall(target_path)

In [ ]:
data_dict = pd.read_excel('/content/drive/MyDrive/Train/data_dict.xlsx')

In [ ]:
#Dicts
d_dict = data_dict[['Demography Table','Unnamed: 1']].dropna()
b_dict = data_dict[['Bureau Data','Unnamed: 4' ]].dropna()
c_dict = data_dict[['CONTRIBUTOR ID', 'Bank Name']].dropna()
a_dict = data_dict[['Account Type', 'Description']].dropna()

In [ ]:
a_dict.shape, b_dict.shape, c_dict.shape, d_dict.shape

((47, 2), (25, 2), (18, 2), (27, 2))

In [ ]:
%timeit

d_train = pd.read_excel('/content/drive/MyDrive/Train/train_Data.xlsx')
b_train = pd.read_excel('/content/drive/MyDrive/Train/train_bureau.xlsx')

d_test = pd.read_excel('/content/drive/MyDrive/test_Data.xlsx')
b_test = pd.read_excel('/content/drive/MyDrive/test_bureau.xlsx')

# Looking at data

In [ ]:
d_train.columns, b_train.columns

(Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
        'BranchID', 'Area', 'Tenure', 'AssetCost', 'AmountFinance',
        'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
        'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
        'MonthlyIncome', 'City', 'State', 'ZiPCODE', 'Top-up Month'],
       dtype='object'),
 Index(['ID', 'SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
        'DATE-REPORTED', 'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'DISBURSED-DT',
        'CLOSE-DT', 'LAST-PAYMENT-DATE', 'CREDIT-LIMIT/SANC AMT',
        'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'INSTALLMENT-FREQUENCY', 'OVERDUE-AMT', 'WRITE-OFF-AMT', 'ASSET_CLASS',
        'REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST',
        'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE'],
       dtype='object'))

In [ ]:
print(same_ids(d_train, b_train), d_train.shape)
print(same_ids(d_test, b_test), d_test.shape)
print(same_ids(d_train, d_test))

128655 (128655, 26)
14745 (14745, 25)
0


In [ ]:
# loo = lookin(d_train)
# loos = lookin(d_test)

# loob = lookin(b_train)
# loobs = lookin(b_test)

# loo.run_this()
# loos.run_this()
# loob.run_this()
# loobs.run_this()

In [ ]:
train_bp = preprocess(b_train)
test_bp = preprocess(b_test)

hist_cols = ['CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
wrap_cols = ['DPD - HIST']

amount_cols = ['CREDIT-LIMIT/SANC AMT','DISBURSED-AMT/HIGH CREDIT',
              'INSTALLMENT-AMT','CURRENT-BAL','OVERDUE-AMT','WRITE-OFF-AMT']
              
date_cols = ['DATE-REPORTED','DISBURSED-DT','CLOSE-DT','DISBURSED-DT']
list_date = ['REPORTED DATE - HIST']

cat_cols = ['MATCH-TYPE', 'ACCT-TYPE', 'ASSET_CLASS',
            'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND', 'ACCOUNT-STATUS',
            ]


In [ ]:

train_bp.history_data(hist_cols,wrap_cols)
train_bp.amount_data(amount_cols)
train_bp.date_data(date_cols, list_date)
train_bp.cat_data(test_bp.df,cat_cols)

test_bp.history_data(hist_cols,wrap_cols)
test_bp.amount_data(amount_cols)
test_bp.date_data(date_cols, list_date)


PREPROCESSING HISTORY DATA
CUR BAL - HIST
AMT OVERDUE - HIST
AMT PAID - HIST
PREPROCESSING AMOUNT DATA
CREDIT-LIMIT/SANC AMT
DISBURSED-AMT/HIGH CREDIT
INSTALLMENT-AMT
CURRENT-BAL
OVERDUE-AMT
WRITE-OFF-AMT
PREPROCESSING DATE DATA
DATE-REPORTED
DISBURSED-DT
CLOSE-DT
DISBURSED-DT
REPORTED DATE - HIST
PREPROCESSING CATEGORICAL DATA
MATCH-TYPE
ACCT-TYPE
ASSET_CLASS
CONTRIBUTOR-TYPE
OWNERSHIP-IND
ACCOUNT-STATUS
PREPROCESSING HISTORY DATA
CUR BAL - HIST
AMT OVERDUE - HIST
AMT PAID - HIST
PREPROCESSING AMOUNT DATA
CREDIT-LIMIT/SANC AMT
DISBURSED-AMT/HIGH CREDIT
INSTALLMENT-AMT
CURRENT-BAL
OVERDUE-AMT
WRITE-OFF-AMT
PREPROCESSING DATE DATA
DATE-REPORTED
DISBURSED-DT
CLOSE-DT
DISBURSED-DT
REPORTED DATE - HIST


In [ ]:
train_dp = preprocess(d_train)
test_dp = preprocess(d_test)
              
date_cols = ['DisbursalDate','MaturityDAte','AuthDate']
list_date = []

cat_cols = ['Frequency','InstlmentMode','LoanStatus',
            'PaymentMode']



In [ ]:
train_dp.date_data(date_cols, list_date)
train_dp.cat_data(test_dp.df,cat_cols)

test_dp.date_data(date_cols, list_date)

PREPROCESSING DATE DATA
DisbursalDate
MaturityDAte
AuthDate
PREPROCESSING CATEGORICAL DATA
Frequency
InstlmentMode
LoanStatus
PaymentMode
PREPROCESSING DATE DATA
DisbursalDate
MaturityDAte
AuthDate


In [ ]:
train_dp.df.isna().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
BranchID               0
Area               11653
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
AssetID                0
ManufacturerID         0
SupplierID             0
LTV                    0
SEX                   59
AGE                   59
MonthlyIncome        234
City               11256
State                  0
ZiPCODE              372
Top-up Month           0
dtype: int64

In [ ]:
# train_dp.df['BranchID'] = train_dp.df['BranchID'].apply(lambda x: str(x)[0:2])
# train_dp.df['BranchID'] = pd.to_numeric(train_dp.df['BranchID'])

# test_dp.df['BranchID'] = test_dp.df['BranchID'].apply(lambda x: str(x)[0:2])
# test_dp.df['BranchID'] = pd.to_numeric(test_dp.df['BranchID'])

In [ ]:
# targetencode(train_dp.df,test_dp.df,['BranchID'],'Top-up Month','mean')

In [ ]:
train_dp.df['Top-up Month'].unique()

array([' > 48 Months', 'No Top-up Service', '12-18 Months',
       '36-48 Months', '18-24 Months', '24-30 Months', '30-36 Months'],
      dtype=object)

In [ ]:
trgt_maps = {' > 48 Months':6, 
             'No Top-up Service':-1, 
             '12-18 Months':1,
              '36-48 Months':5, 
              '18-24 Months':2, 
              '24-30 Months':3, 
              '30-36 Months':4}

In [ ]:
train_dp.df['Top-up Month'] = train_dp.df['Top-up Month'].map(trgt_maps)
# test_dp.df['Top-up Month'] = test_dp.df['Top-up Month'].map(trgt_maps)

In [ ]:
train_dp.df['Top-up Month']

0         6
1        -1
2         1
3         6
4         5
         ..
128650    3
128651   -1
128652   -1
128653   -1
128654   -1
Name: Top-up Month, Length: 128655, dtype: int64

In [ ]:
train_dp.df.nunique()

ID                 128655
Frequency               4
InstlmentMode           2
LoanStatus              2
PaymentMode            11
BranchID              189
Area                   92
Tenure                141
AssetCost            7835
AmountFinance       20439
DisbursalAmount     19412
EMI                 24323
DisbursalDate        2837
MaturityDAte          980
AuthDate             2711
AssetID            128655
ManufacturerID         10
SupplierID           4539
LTV                  7988
SEX                     2
AGE                    73
MonthlyIncome       11568
City                  488
State                  22
ZiPCODE              9123
Top-up Month            7
dtype: int64

In [ ]:
train_dp.df,test_dp.df = targetencode(train_dp.df,test_dp.df,
             ['PaymentMode','BranchID','ManufacturerID','State'],'Top-up Month','mean')

PaymentMode
BranchID
ManufacturerID
State


In [ ]:
train_dp.df

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,2,1,1,-0.213344,-0.159420,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,-0.168199,21946,61.11,M,49.0,35833.33,RAISEN,0.059931,464993.0,6
1,2,2,0,1,-0.058121,0.236842,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,-0.044586,34802,70.00,M,23.0,666.67,SEHORE,0.059931,466001.0,-1
2,3,3,1,0,-0.052446,-0.159420,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,-0.107265,127335,69.77,M,39.0,45257.00,BHOPAL,0.059931,462030.0,1
3,7,2,0,1,-0.180246,0.001541,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,-0.107265,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,0.059931,473335.0,6
4,8,2,1,1,-0.180246,-0.316129,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,-0.117570,21853,71.05,M,56.0,27313.67,BILASPUR,-0.406021,495442.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128650,143390,1,1,1,-0.052446,-0.368421,PANIPAT,24,470000,265601.0,265601.0,76800.0,2018-09-21,2020-06-05,2018-09-22,31286914,-0.168199,48879,40.17,M,25.0,65333.33,SONIPAT,-0.090728,131403.0,3
128651,143391,1,1,1,-0.052446,-0.368421,PANIPAT,24,460000,275630.0,275630.0,80100.0,2018-09-22,2020-06-05,2018-09-22,31295422,-0.168199,48879,59.92,M,25.0,83333.33,SONIPAT,-0.090728,131403.0,-1
128652,143393,2,1,0,-0.052446,-0.368421,PANIPAT,23,545000,300733.0,300733.0,15277.0,2018-11-23,2020-11-05,2018-11-23,32145629,-0.168199,44118,52.38,M,36.0,248500.00,SONIPAT,-0.090728,131024.0,-1
128653,143394,1,1,0,-0.052446,-0.368421,PANIPAT,35,350000,250962.0,250962.0,74341.0,2018-12-20,2021-06-05,2018-12-20,32509866,-0.168199,48879,50.37,M,37.0,84500.00,SONIPAT,-0.090728,131103.0,-1


In [ ]:
train_dp.df = train_dp.df.drop(columns = ['City','Area','ZiPCODE','SupplierID','AssetID'])
test_dp.df = test_dp.df.drop(columns = ['City','Area','ZiPCODE','SupplierID','AssetID'])

In [ ]:
train_bp.df

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,0,38,6,2018-04-30,1,3,2015-10-05,NaT,2018-02-27,NaN,37352.0,NaN,37873.0,NaN,37873.0,0.0,6,"[2018-04-30 00:00:00, 2018-03-31 00:00:00]","[030, 000]","[37873.0, 12820.0]","[37873.0, 0]","[0, 0]",NaN
1,1,False,0,0,6,2019-12-31,1,0,2018-03-19,NaT,2019-12-19,NaN,44000.0,1405.0,20797.0,F03,NaN,0.0,6,"[2019-12-31 00:00:00, 2019-11-30 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 000, 000, 000, ...","[20797.0, 21988.0, 23174.0, 24341.0, 25504.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36.0
2,1,True,0,48,7,2020-01-31,1,0,2019-08-30,NaT,NaT,NaN,145000.0,NaN,116087.0,NaN,0.0,0.0,8,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...","[000, 000, 000, 000, 000, 000]","[116087.0, 116087.0, 145000.0, 145000.0, 14500...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0]",NaN
3,1,True,0,0,7,2017-09-30,1,2,2013-09-27,2017-09-21,NaT,NaN,300000.0,NaN,0.0,NaN,0.0,0.0,8,"[2017-09-30 00:00:00, 2017-08-01 00:00:00, 201...","[000, DDD, 027, 026, 027, 026, 027, 024, 027, ...","[0.0, 0, 15925.0, 23754.0, 31494.0, 39147.0, 4...","[0.0, 0, 1014.0, 1014.0, 1014.0, 1014.0, 1014....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
4,1,True,0,48,7,2016-02-29,1,2,2012-02-10,2016-02-01,NaT,NaN,275000.0,NaN,0.0,NaN,0.0,0.0,8,"[2016-02-29 00:00:00, 2016-01-31 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 000, 000, 000, ...","[0.0, 0.0, 23658.0, 23321.0, 22989.0, 46321.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560839,143393,False,0,48,7,2019-12-31,1,2,NaT,NaT,NaT,NaN,250000.0,NaN,0.0,NaN,0.0,0.0,6,"[2017-12-31 00:00:00, 2017-11-30 00:00:00, 201...","[000, 000, DDD, 054, 024, 024, 000, 000, 000, ...","[0.0, 0.0, 0, 67747.0, 89638.0, 111197.0, 1095...","[0.0, 0.0, 0, 22920.0, 22920.0, 22920.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",12.0
560840,143393,False,0,48,7,2019-12-31,1,0,2017-10-31,NaT,NaT,NaN,30000.0,NaN,0.0,NaN,0.0,0.0,6,"[2019-12-31 00:00:00, 2019-11-30 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 027, 028, 028, ...","[0.0, 0.0, 14208.0, 28265.0, 42149.0, 55874.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 125.0, 125.0, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",24.0
560841,143393,False,0,0,7,2020-01-31,0,0,2016-11-28,NaT,2020-01-16,NaN,393819.0,NaN,11687.0,NaN,NaN,0.0,6,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...","[XXX, XXX, XXX, XXX, XXX, XXX, XXX, XXX, XXX, ...","[101687.0, 111161.0, 120514.0, 129749.0, 13886...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10897.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",48.0
560842,143394,True,0,48,7,2020-01-31,1,0,2018-12-20,NaT,NaT,NaN,250643.0,NaN,132487.0,NaN,0.0,0.0,8,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...",[0],"[132487.0, 132487.0, 132487.0, 191426.0, 19142...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN


In [ ]:
train_bp.df = train_bp.df.drop(columns  = ['INSTALLMENT-AMT',
                                           'CREDIT-LIMIT/SANC AMT',
                                           'INSTALLMENT-AMT',
                                           'INSTALLMENT-FREQUENCY'])
test_bp.df = test_bp.df.drop(columns  = ['INSTALLMENT-AMT',
                                           'CREDIT-LIMIT/SANC AMT',
                                           'INSTALLMENT-AMT',
                                           'INSTALLMENT-FREQUENCY'])

In [ ]:
train_bp.df['SELF-INDICATOR'] = train_bp.df['SELF-INDICATOR'].astype('int64')

In [ ]:
train_dp.df['SEX'].value_counts()

M    122144
F      6452
Name: SEX, dtype: int64

In [ ]:
encoder = LabelEncoder()
train_dp.df['SEX'] = encoder.fit_transform(train_dp.df['SEX'].fillna('M'))
test_dp.df['SEX'] = encoder.fit_transform(test_dp.df['SEX'].fillna('M'))

In [ ]:
# # Finding out active and LTFS
# ltfs_trb = train_bp.df[train_bp.df['SELF-INDICATOR']==1]
# ltfs_tsb = train_bp.df[train_bp.df['SELF-INDICATOR']==1]

In [ ]:
# ltfs_trb.shape, train_dp.df.shape[0] + test_dp.df.shape[0]

In [ ]:
# ltfs_trm = ltfs_trb.merge(train_dp.df,
#                left_on = ['ID','DISBURSED-AMT/HIGH CREDIT'],
#                right_on = ['ID','DisbursalAmount'],
#                how = 'inner')

In [ ]:
train_bp.df = train_bp.df.merge(train_dp.df[['ID','Top-up Month']],on='ID',how='left')

In [ ]:
train_bp.df,test_bp.df = targetencode(train_bp.df,test_bp.df,
             ['ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','ASSET_CLASS'],'Top-up Month','mean')

ACCT-TYPE
CONTRIBUTOR-TYPE
OWNERSHIP-IND
ACCOUNT-STATUS
ASSET_CLASS


In [ ]:
train_bp.df

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,DISBURSED-AMT/HIGH CREDIT,CURRENT-BAL,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,Top-up Month
0,1,0,0,2.239859,1.856057,2018-04-30,1.458272,1.148350,2015-10-05,NaT,2018-02-27,37352.0,37873.0,37873.0,0.0,1.792203,"[2018-04-30 00:00:00, 2018-03-31 00:00:00]","[030, 000]","[37873.0, 12820.0]","[37873.0, 0]","[0, 0]",NaN,6
1,1,0,0,1.194759,1.856057,2019-12-31,1.458272,1.384176,2018-03-19,NaT,2019-12-19,44000.0,20797.0,NaN,0.0,1.792203,"[2019-12-31 00:00:00, 2019-11-30 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 000, 000, 000, ...","[20797.0, 21988.0, 23174.0, 24341.0, 25504.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36.0,6
2,1,1,0,0.963488,1.071563,2020-01-31,1.458272,1.384176,2019-08-30,NaT,NaT,145000.0,116087.0,0.0,0.0,1.135800,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...","[000, 000, 000, 000, 000, 000]","[116087.0, 116087.0, 145000.0, 145000.0, 14500...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0]",NaN,6
3,1,1,0,1.194759,1.071563,2017-09-30,1.458272,1.491686,2013-09-27,2017-09-21,NaT,300000.0,0.0,0.0,0.0,1.135800,"[2017-09-30 00:00:00, 2017-08-01 00:00:00, 201...","[000, DDD, 027, 026, 027, 026, 027, 024, 027, ...","[0.0, 0, 15925.0, 23754.0, 31494.0, 39147.0, 4...","[0.0, 0, 1014.0, 1014.0, 1014.0, 1014.0, 1014....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,6
4,1,1,0,0.963488,1.071563,2016-02-29,1.458272,1.491686,2012-02-10,2016-02-01,NaT,275000.0,0.0,0.0,0.0,1.135800,"[2016-02-29 00:00:00, 2016-01-31 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 000, 000, 000, ...","[0.0, 0.0, 23658.0, 23321.0, 22989.0, 46321.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560839,143393,0,0,0.963488,1.071563,2019-12-31,1.458272,1.491686,NaT,NaT,NaT,250000.0,0.0,0.0,0.0,1.792203,"[2017-12-31 00:00:00, 2017-11-30 00:00:00, 201...","[000, 000, DDD, 054, 024, 024, 000, 000, 000, ...","[0.0, 0.0, 0, 67747.0, 89638.0, 111197.0, 1095...","[0.0, 0.0, 0, 22920.0, 22920.0, 22920.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",12.0,0
560840,143393,0,0,0.963488,1.071563,2019-12-31,1.458272,1.384176,2017-10-31,NaT,NaT,30000.0,0.0,0.0,0.0,1.792203,"[2019-12-31 00:00:00, 2019-11-30 00:00:00, 201...","[000, 000, 000, 000, 000, 000, 027, 028, 028, ...","[0.0, 0.0, 14208.0, 28265.0, 42149.0, 55874.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 125.0, 125.0, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",24.0,0
560841,143393,0,0,1.194759,1.071563,2020-01-31,1.240538,1.384176,2016-11-28,NaT,2020-01-16,393819.0,11687.0,NaN,0.0,1.792203,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...","[XXX, XXX, XXX, XXX, XXX, XXX, XXX, XXX, XXX, ...","[101687.0, 111161.0, 120514.0, 129749.0, 13886...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10897.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",48.0,0
560842,143394,1,0,0.963488,1.071563,2020-01-31,1.458272,1.384176,2018-12-20,NaT,NaT,250643.0,132487.0,0.0,0.0,1.135800,"[2020-01-31 00:00:00, 2019-12-31 00:00:00, 201...",[0],"[132487.0, 132487.0, 132487.0, 191426.0, 19142...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,0


In [ ]:
bp_tgg = train_bp.df.groupby('ID').aggregate({
    'ACCT-TYPE':'mean',
    'CONTRIBUTOR-TYPE':'mean',
    'OWNERSHIP-IND':'mean',
    'ACCOUNT-STATUS':'mean',
    'ASSET_CLASS':'mean',
    
    'SELF-INDICATOR': 'sum',
    'REPORTED DATE - HIST': 'sum',
    'DPD - HIST': 'sum',
    'CUR BAL - HIST': 'sum',
    'AMT OVERDUE - HIST': 'sum',
    'AMT PAID - HIST':'sum',
    'TENURE':'mean'

}).reset_index()

In [ ]:
bp_tsgg = test_bp.df.groupby('ID').aggregate({
    
     'ACCT-TYPE':'mean',
    'CONTRIBUTOR-TYPE':'mean',
    'OWNERSHIP-IND':'mean',
    'ACCOUNT-STATUS':'mean',
    'ASSET_CLASS':'mean',
    'SELF-INDICATOR': 'sum',
    'REPORTED DATE - HIST': 'sum',
    'DPD - HIST': 'sum',
    'CUR BAL - HIST': 'sum',
    'AMT OVERDUE - HIST': 'sum',
    'AMT PAID - HIST':'sum',
    'TENURE':'mean'

}).reset_index()

bp_tgg = bp_tgg.merge(train_dp.df,on='ID')
bp_tsgg = bp_tsgg.merge(test_dp.df,on='ID')

bp_tgg['DURATION'] = bp_tgg['MaturityDAte']- bp_tgg['DisbursalDate']
bp_tsgg['DURATION'] = bp_tsgg['MaturityDAte']- bp_tsgg['DisbursalDate']

In [ ]:
# train_dp.df['Duration'] = train_dp.df['DisbursalDate'] - train_dp.df['MaturityDAte']
# test_dp.df['Duration'] = test_dp.df['DisbursalDate'] - test_dp.df['MaturityDAte']

# train_dp.df['Duration'] = train_dp.df['Duration'].apply(lambda x: x.days)
# test_dp.df['Duration'] = test_dp.df['Duration'].apply(lambda x: x.days)

# train_dp.df = train_dp.df.drop(columns = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','ManufacturerID','SupplierID',
#                                           'BranchID','State'])
# test_dp.df = test_dp.df.drop(columns = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','ManufacturerID','SupplierID',
#                                         'BranchID','State'])


In [ ]:
bp_tgg['lens'] = bp_tgg[['REPORTED DATE - HIST',
        'CUR BAL - HIST']].apply(lambda x: min(len(x['REPORTED DATE - HIST']),
                                                   len(x['CUR BAL - HIST'])),axis=1)
        
bp_tgg['REPORTED DATE - HIST'] = bp_tgg.apply(lambda x: x['REPORTED DATE - HIST'][:x['lens']],axis=1)
bp_tgg['CUR BAL - HIST'] = bp_tgg.apply(lambda x: x['CUR BAL - HIST'][:x['lens']],axis=1)
bp_tgg['CUR BAL - HIST'] = bp_tgg.apply(lambda x: x['CUR BAL - HIST'][:x['lens']],axis=1)

In [ ]:
# en = LabelEncoder()
# en.fit(train_dp.df['Top-up Month'])
# train_dp.df['Top-up Month'] = en.transform(train_dp.df['Top-up Month'])

In [ ]:
bp_tsgg['lens'] = bp_tsgg[['REPORTED DATE - HIST',
        'CUR BAL - HIST']].apply(lambda x: min(len(x['REPORTED DATE - HIST']),
                                                   len(x['CUR BAL - HIST'])),axis=1)
        
bp_tsgg['REPORTED DATE - HIST'] = bp_tsgg.apply(lambda x: x['REPORTED DATE - HIST'][:x['lens']],axis=1)
bp_tsgg['CUR BAL - HIST'] = bp_tsgg.apply(lambda x: x['CUR BAL - HIST'][:x['lens']],axis=1)
bp_tsgg['CUR BAL - HIST'] = bp_tsgg.apply(lambda x: x['CUR BAL - HIST'][:x['lens']],axis=1)

In [ ]:
bp_tgg.columns

Index(['ID', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND',
       'ACCOUNT-STATUS', 'ASSET_CLASS', 'SELF-INDICATOR',
       'REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST',
       'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE', 'Frequency',
       'InstlmentMode', 'LoanStatus', 'PaymentMode', 'BranchID', 'Tenure',
       'AssetCost', 'AmountFinance', 'DisbursalAmount', 'EMI', 'DisbursalDate',
       'MaturityDAte', 'AuthDate', 'ManufacturerID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'State', 'Top-up Month', 'DURATION', 'lens'],
      dtype='object')

In [ ]:
bp_tgg1= bp_tgg[['ID','EMI','REPORTED DATE - HIST','CUR BAL - HIST']].set_index(['ID']).apply(pd.Series.explode).reset_index()
bp_tsgg1= bp_tsgg[['ID','EMI','REPORTED DATE - HIST','CUR BAL - HIST']].set_index(['ID']).apply(pd.Series.explode).reset_index()

In [ ]:
bp_tgg1 = bp_tgg1.merge(bp_tgg[['ID', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND',
       'ACCOUNT-STATUS', 'ASSET_CLASS', 'SELF-INDICATOR',
       'TENURE', 'Frequency',
       'InstlmentMode', 'LoanStatus', 'PaymentMode', 'BranchID', 'Tenure',
       'AssetCost', 'AmountFinance', 'DisbursalAmount', 'DisbursalDate',
       'MaturityDAte', 'AuthDate', 'ManufacturerID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'State', 'Top-up Month', 'DURATION', 'lens']],on='ID')

bp_tsgg1 = bp_tsgg1.merge(bp_tsgg[['ID', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND',
       'ACCOUNT-STATUS', 'ASSET_CLASS', 'SELF-INDICATOR',
       'TENURE', 'Frequency',
       'InstlmentMode', 'LoanStatus', 'PaymentMode', 'BranchID', 'Tenure',
       'AssetCost', 'AmountFinance', 'DisbursalAmount', 'DisbursalDate',
       'MaturityDAte', 'AuthDate', 'ManufacturerID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'State', 'DURATION', 'lens']],on='ID')

In [ ]:
bp_tgg1.to_csv('/content/drive/MyDrive/pro_train_ltfs1.csv',index = None)
bp_tsgg1.to_csv('/content/drive/MyDrive/pro_test_ltfs1.csv',index = None)